## Convert the R1CS into a QAP


## Over Real Number

In [121]:
import numpy as np
import random

# Define the matrices
L = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

R = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

O = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [-3,1,1,2,0,-1]])

# pick random values for x and y
x = random.randint(1,1000)
y = random.randint(1,1000)

# this is our orignal formula
out = 3 * x * x * y + 5 * x * y - x- 2*y + 3# the witness vector with the intermediate variables inside
v1 = 3*x*x
v2 = v1 * y
w = np.array([1, out, x, y, v1, v2])

print("witness {}".format(w))

result = O.dot(w) == np.multiply(L.dot(w),R.dot(w))
assert result.all(), "result contains an inequality"

witness [        1 318774976       755       186   1710075 318073950]


In [123]:
from scipy.interpolate import lagrange
import numpy as np
from numpy.polynomial import polynomial as P

np.set_printoptions(precision=2)


def poly_interpolate(index, matrics):
    res = []
    for row in matrics:
        poly = lagrange(index, row).coef[::-1]
        res.append(poly)
    return np.array(res, dtype=object)


def polymuln(lst):
    if len(lst) == 0:
        return 1
    return P.polymul(lst[0], polymuln(lst[1:]))


def QAP(s, L, R, O):
    U = np.matmul(s, L)
    V = np.matmul(s, R)
    W = np.matmul(s, O)
    print(f"L: {L}")
    print(f"R: {R}")
    print(f"O: {O}")
    Q = P.polysub(P.polymul(U, V), W)
    print(f"T:\n {Q}")
    assert len(U) == len(V) == len(W)
    return P.polydiv(Q, Z(len(U)))


def Z(n):
    ns = [[-(1 + i), 1] for i, _ in enumerate(range(n))]
    return polymuln(ns)


def r1cs_to_qap_real(s, L, R, O):
    x = np.arange(1, len(L) + 1)

    resL = poly_interpolate(x, np.transpose(L))
    resR = poly_interpolate(x, np.transpose(R))
    resO = poly_interpolate(x, np.transpose(O))

    print(f"interporated L: \n {resL}")
    print(f"interporated R: \n {resR}")
    print(f"interporated O: \n {resO}")

    return QAP(s, resL, resR, resO)



print(r1cs_to_qap_real(w, L, R, O))

interporated L: 
 [array([0.]) array([0.]) array([10., -9.,  2.]) array([0.])
 array([-3.,  4., -1.]) array([0.])]
interporated R: 
 [array([0.]) array([0.]) array([ 3. , -2.5,  0.5])
 array([ 2. , -3.5,  1.5]) array([0.]) array([0.])]
interporated O: 
 [[-3.0 4.5 -1.5]
 [1.0 -1.5 0.5]
 [1.0 -1.5 0.5]
 [2.0 -3.0 1.0]
 [3.0 -2.5 0.5]
 [-4.0 5.5 -1.5]]
L: [array([0.]) array([0.]) array([10., -9.,  2.]) array([0.])
 array([-3.,  4., -1.]) array([0.])]
R: [array([0.]) array([0.]) array([ 3. , -2.5,  0.5])
 array([ 2. , -3.5,  1.5]) array([0.]) array([0.])]
O: [[-3.0 4.5 -1.5]
 [1.0 -1.5 0.5]
 [1.0 -1.5 0.5]
 [2.0 -3.0 1.0]
 [3.0 -2.5 0.5]
 [-4.0 5.5 -1.5]]
T:
 [-12560104500.0 29756895785.0 -24898506647.5 8823388285.0 -1121672922.5]
(array([2093350750.0, -1121672922.5], dtype=object), array([0.0], dtype=object))


## Over Finite Field

In [124]:
import numpy as np
import random
import galois
P = 79
GF = galois.GF(P)

L_F = np.array([[0,0,3,0,0,0],
               [0,0,0,0,1,0],
               [0,0,1,0,0,0]])

R_F = np.array([[0,0,1,0,0,0],
               [0,0,0,1,0,0],
               [0,0,0,5,0,0]])

O_F = np.array([[0,0,0,0,1,0],
               [0,0,0,0,0,1],
               [GF(P-3),1,1,2,0,GF(P-1)]])

L_galois = GF(L_F)
R_galois = GF(R_F)
O_galois = GF(O_F)

# pick random values for x and y
x = GF(1)
y = GF(2)


# this is our original formula
v1 = 3*x*x
v2 = v1 * y
out = 3*x*x*y + 5*x*y + GF(P-1)*x + GF(P-2)*y + GF(3)
# the witness vector with the intermediate variables inside
w = GF(np.array([1, out, x, y, v1, v2]))

print("witness {}".format(w))

result = O_galois.dot(w) == np.multiply(L_galois.dot(w),R_galois.dot(w))
assert result.all(), "result contains an inequality"

witness [ 1 14  1  2  3  6]


In [146]:
from scipy.interpolate import lagrange
import numpy as np
import functools 
from numpy.polynomial import polynomial as P

import galois
P = 79
GF = galois.GF(P)

np.set_printoptions(precision=2)


def poly_finite_field_interpolate(index, matrics):
    res = []
    for row in matrics:
        poly = galois.lagrange_poly(index, row)
        coef = poly.coefficients()[::-1]
        res.append(coef)
    return np.array(res, dtype=object)


def polymuln(lst):
    if len(lst) == 0:
        return 1
    return P.polymul(lst[0], polymuln(lst[1:]))

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return functools.reduce(sum_, map(mul_, polys, witness))


def QAP(s, L, R, O):

    U = inner_product_polynomials_with_witness(s, L)
    V = inner_product_polynomials_with_witness(s, R)

    W_int = np.array(O, dtype=int)    
    W = inner_product_polynomials_with_witness(s, GF(W_int))

    print(f"U: {galois.Poly(U[::-1])}")
    print(f"V: {galois.Poly(V[::-1])}")
    print(f"W: {galois.Poly(W[::-1])}")

    T = galois.Poly([1, P-1], field=GF)
    for i in range(2, U.shape[0] + 1):
        T *= galois.Poly([1, P-i], field=GF)

    print("\nT = ", T)

    H = (galois.Poly(U[::-1]) * galois.Poly(V[::-1]) - galois.Poly(W[::-1])) // T
    print("\nH = ",H)
    rem = (galois.Poly(U[::-1]) * galois.Poly(V[::-1]) - galois.Poly(W[::-1])) % T
    print("\nrem = ", rem)
    print("\nh(x)t(x = ", H*T)

    assert rem == 0
    assert galois.Poly(U[::-1]) * galois.Poly(V[::-1]) == galois.Poly(W[::-1]) + H * T, "division has a remainder"
    assert len(U) == len(V) == len(W)
    return H, T


def r1cs_to_qap_field(s, L, R, O):
    x = GF(np.arange(1, len(L) + 1))

    L_galois = GF(L)
    R_galois = GF(R)
    O_galois = GF(O)

    resL = poly_finite_field_interpolate(x, np.transpose(L_galois))
    resR = poly_finite_field_interpolate(x, np.transpose(R_galois))
    resO = poly_finite_field_interpolate(x, np.transpose(O_galois))

    print(f"interporated L: \n {resL}")
    print(f"interporated R: \n {resR}")
    print(f"interporated O: \n {resO}")

    return QAP(s, resL, resR, resO)


In [145]:
from py_ecc.optimized_bn128 import multiply, G1, G2, add, pairing, neg, normalize

H, T = r1cs_to_qap_field(w, L_F, R_F, O_F)

print("\nH = ",H)
print("\nT = ",T)

print("Setup phase")
print("-"*10)
print("Toxic waste:")

tau = GF(20)
T_tau = T(tau)
print(f"\nT(τ) = {T_tau}")

for i in range(1, T.degree + 2):
    print(f"T({i}) = ", T(i))
    if i == T.degree:
        print("-"*10)

interporated L: 
 [GF([0], order=79) GF([0], order=79) GF([10, 70,  2], order=79)
 GF([0], order=79) GF([76,  4, 78], order=79) GF([0], order=79)]
interporated R: 
 [GF([0], order=79) GF([0], order=79) GF([ 3, 37, 40], order=79)
 GF([ 2, 36, 41], order=79) GF([0], order=79) GF([0], order=79)]
interporated O: 
 [[76 44 38]
 [1 38 40]
 [1 38 40]
 [2 76 1]
 [3 37 40]
 [75 45 38]]
U: 78x^2 + 3x + 1
V: 43x^2 + 30x + 7
W: 40x^2 + 41x + 1

T =  x^3 + 73x^2 + 11x + 73

H =  36x + 78

rem =  0

h(x)t(x =  36x^4 + 20x^3 + 7x^2 + 10x + 6

U.shape[0 =  3
T(1) =  0
T(2) =  0
T(3) =  0
----------
T(4) =  6

T(τ) = 47

H =  36x + 78

T =  x^3 + 73x^2 + 11x + 73
T(1) =  0
T(2) =  0
T(3) =  0
----------
T(4) =  6

T(τ) = 47
